# Part 1. Load, Prep, Train, Register, Deploy and Scale in 50 Lines of Code

In this lab you will learn how to:

1. Create a session for Snowpark with Snowflake
2. Create a DB, Warehouse and Model Registry
3. Prep Data using the highly parallelisable vectorised UDTF functionality
4. Build/train a regression model with Snowpark ML
5. Register your model in the Model Registry
6. Run the model

All this in 50 lines of code (less the library imports). 

### Note - there are some "TO DOs" along the way for you to update, they are signposted with markdowns and comments, so keep your eyes open

## Prerequisites:
In a terminal please run:

conda env create -f conda_env.yml
 
conda activate snowpark-ml-hol

jupyter lab <---- this will load jupyter (you cna execute the notebook anywhere really, e.g. vscode, but jupyter is an easy option)

# 1.0 Imports
TO DO: just run the following cell

In [1]:
import json
import pandas as pd
from snowflake.snowpark.session import Session
import snowflake.snowpark.functions as F
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, DateType
from snowflake.ml.modeling.xgboost import XGBRegressor
from snowflake.ml.modeling.linear_model import LinearRegression
from snowflake.ml.registry import registry
from snowflake.ml._internal.utils import identifier

# 1.1 Reading Snowflake Connection Details, create a Session

TO DO: 

1. Create a JSON with your credentials and update the cell below

{
"account": "your_account_name", 
"user": "your_user_name",
"password": "insert_your_pwd_here",
"role": "ACCOUNTADMIN"
}

2. Update the location 

In [2]:
snowflake_connection_cfg = json.loads(open("creds.json").read()) # <--- 2. Update here if not in the current folder
session = Session.builder.configs(snowflake_connection_cfg).create()

# 1.2 Specify Your Database, Create a Virtual Warehouse and Load Your Data

Snowflake seperates compute from storage, so we need a database AND a warehouse (compute environment) to run this stuff on.  A model registry is just a special type of database, and while you could use the same database as your data, in this case we're using a distinct database for simplicity.

In [3]:
session.sql("CREATE OR REPLACE DATABASE MODEL_REGISTRY").collect()
session.sql("CREATE OR REPLACE SCHEMA PUBLIC").collect()
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY"
REGISTRY_SCHEMA_NAME = "PUBLIC"
native_registry = registry.Registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)
session.sql("CREATE OR REPLACE DATABASE HOL_DEMO").collect()
session.sql("CREATE OR REPLACE WAREHOUSE ASYNC_WH WITH WAREHOUSE_SIZE='MEDIUM' WAREHOUSE_TYPE = 'SNOWPARK-OPTIMIZED'").collect()

In [4]:
# This data load is deliberately lo-fi, lots of other ways of importing data exist that have greater scale, but this compact approach is fine for this task
session.write_pandas(pd.read_csv("test.csv"), table_name='FS_DATASET', auto_create_table=True, overwrite=True)

# 1.3 Get Your Data (Prepped) With Pandas and Snowpark
We're got time series data, and we want to munge the data into a regression friendly dataset.  Pandas is a common approach for this in data science, and while greater performance can be achieved with Snowpark native manipulations, vectorised UDTFs offer a great combination of familiarity (i.e. do the same thing you would do in pandas) and scale (parallelisation partitioned across each symbol)

TO DO: just run the following cells

In [5]:
sdf = session.table("FS_DATASET")
sdf = sdf.select(F.to_date(F.col('DATE')).as_('DATE'), "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL")

In [6]:
class ML_Prep:
    def end_partition(self, df):
        df.columns = ['_DATE', "_OPEN", "_HIGH", "_LOW", "_CLOSE", "_SYMBOL"]
        for i in range(1,6):
            df["_CLOSE-" + str(i)] = df["_CLOSE"].shift(i).bfill()
        yield df

ML_Prep.end_partition._sf_vectorized_input = pd.DataFrame

ml_prep_udtf = session.udtf.register(
    ML_Prep, # the class
    name="ml_prep_udtf",
    input_types=[PandasDataFrameType([DateType(), FloatType(), FloatType(), FloatType(), FloatType(), StringType()])], 
    output_schema=PandasDataFrameType([DateType(), FloatType(), FloatType(), FloatType(), FloatType(), StringType(),FloatType(),FloatType(),FloatType(),FloatType(),FloatType(),FloatType()],
                                      ['DATE', "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL", "CLOSE_M1", "CLOSE_M2", "CLOSE_M3", "CLOSE_M4", "CLOSE_M5"]),
    packages=["snowflake-snowpark-python", 'pandas'])  


In [7]:
sdf_prepped = sdf.select(ml_prep_udtf(*["DATE", "OPEN", "HIGH", "LOW", "CLOSE", "SYMBOL"]).over(partition_by=['SYMBOL']))
sdf_prepped.limit(10).to_pandas()
sdf_prepped.write.save_as_table("ML_PREPPED", mode="overwrite")
sdf[['SYMBOL']].distinct().to_pandas()

# 1.4.1 Choose Your Symbol, Train/Test Split and Model

We've got our data ready, but we need to make a few selections before we build our models

TO DO:
1. Choose the Symbol you want to build a model for
2. Pick the date range for your train/test split
3. Pick a regression model you want type

In [9]:
sdf_prepped_filt = sdf_prepped.filter((F.col("SYMBOL") == "")) # <---- update 1.
sdf_filt_train, sdf_filt_test = sdf_prepped_filt.filter((F.col("DATE") <= '')), sdf_prepped_filt.filter((F.col("DATE") > '')) # <---- update 2.
regressor = # <---- update 3. hint one look at our imports cell

# 1.4.2 Train Your Model

Our model is almost ready to be trained, but we need to choose our inputs, targets, and outputs.  We could go off piste and alter model (hyper)parameters here too (https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/api/modeling/snowflake.ml.modeling.linear_model.LinearRegression)

TO DO:
1. Select your input columns
2. Select your target(label) column
3. Choose your output column name

In [10]:
regressor = regressor(input_cols=[], # <---- update 1.
                         label_cols=[], # <---- update 2.
                         output_cols=[]) # <---- update 3.
regressor.fit(sdf_prepped_filt)

# 1.5 Register Your Model

Let's assume we love the first model, it's time to register it....

TO DO:
1. Choose a  model name
2. Choose a model version (note the combo of name and version needs to be unique)

In [11]:
MODEL_NAME = # <---- update 1. use a string like "REGRESSION_MODEL"
MODEL_VERSION = # <---- update 2. use a string like "v1"

model = native_registry.log_model(
    model_name=MODEL_NAME,
    version_name=MODEL_VERSION,
    model=regressor,
)

# 1.6 Run Your Model

We're at the finish line!

TO DO: just run the following cell

In [ ]:
model.run(sdf_filt_test, function_name="predict").limit(20).to_pandas()

## Make sure you run this last line - it's needed for the Part 2. and 3. of the Lab

In [ ]:
model_ = native_registry.get_model(MODEL_NAME).version(MODEL_VERSION)
model_.run(sdf_filt_test, function_name="predict").write.save_as_table("ML_PREDICT", mode="overwrite")